## PostClass 

In [123]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (10, 6)
pd.set_option("display.max_columns", 100)

In [124]:
df = pd.read_csv("Advertising.csv")
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [125]:
X = df.drop(columns="sales")
y = df["sales"]

### Polynomial Features

In [126]:
from sklearn.preprocessing import PolynomialFeatures

In [127]:
polynomial_converter = PolynomialFeatures(degree=5, include_bias=False)
poly_features = polynomial_converter.fit_transform(X)

In [128]:
poly_features.shape

(200, 55)

### Train | Test Split 

In [129]:
from sklearn.model_selection import train_test_split

In [130]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

### Scaling The Data

In [131]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
# RobustScaler is used when outlier could be present.

scaler = StandardScaler()

In [132]:
scaler.fit(X_train) # Apply just for X_train not for X_test

StandardScaler()

In [133]:
X_train_scaled = scaler.transform(X_train)
X_train_scaled

array([[ 0.49300171, -0.33994238,  1.61586707, ...,  0.00704484,
         0.36731969,  0.73567879],
       [ 1.61337153,  1.32852213,  1.90079242, ...,  2.67449856,
         2.0763226 ,  1.3268794 ],
       [-0.84811893, -1.58789957, -1.02319564, ..., -0.42823202,
        -0.36509041, -0.32221291],
       ...,
       [ 1.04606563, -1.30982215,  0.27064572, ..., -0.42440306,
        -0.34885122, -0.26314509],
       [ 0.74817069,  0.03987068, -1.26608283, ..., -0.4280649 ,
        -0.36507729, -0.32226899],
       [ 0.13813882,  0.55533126,  1.01799092, ...,  0.24192258,
         0.16357974,  0.0357574 ]])

In [134]:
X_test_scaled = scaler.transform(X_test)
X_test_scaled

array([[-0.91339472,  1.76259419,  0.68168558, ...,  0.62770088,
         0.09523693, -0.14999627],
       [ 1.23121149,  0.23655959, -1.19601922, ..., -0.42768623,
        -0.36503783, -0.32226552],
       [-0.46002473, -0.40776614,  0.35005115, ..., -0.34929255,
        -0.2825677 , -0.24821148],
       ...,
       [-1.33115973,  1.19965698, -1.18200649, ..., -0.42673785,
        -0.36499105, -0.3222642 ],
       [ 1.3522684 , -0.23142436, -1.4389064 , ..., -0.42823197,
        -0.36509041, -0.32226987],
       [ 0.27818504, -0.06186496,  0.01841672, ..., -0.35649712,
        -0.31776442, -0.29546644]])

### Polynomial Regression Model Building 

In [135]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()

In [136]:
lm.fit(X_train_scaled, y_train) # y_train scale edilmiyor çünkü tek bir feature'dan oluşuyor.

LinearRegression()

In [137]:
y_pred = lm.predict(X_test_scaled)
y_train_pred = lm.predict(X_train_scaled)

In [138]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def train_val(y_train, y_train_pred, y_test, y_pred, name):
    
    scores = {name+"_train": {"R2" : r2_score(y_train, y_train_pred),
    "mae" : mean_absolute_error(y_train, y_train_pred),
    "mse" : mean_squared_error(y_train, y_train_pred),                          
    "rmse" : np.sqrt(mean_squared_error(y_train, y_train_pred))},
    
    name+"_test": {"R2" : r2_score(y_test, y_pred),
    "mae" : mean_absolute_error(y_test, y_pred),
    "mse" : mean_squared_error(y_test, y_pred),
    "rmse" : np.sqrt(mean_squared_error(y_test, y_pred))}}
    
    return pd.DataFrame(scores)

In [139]:
ls = train_val(y_train, y_train_pred, y_test, y_pred, "linear")
ls
# Train R2_score u test score'una göre çok yüksek. Burada bir overfitting durumu var.
# Zaten polynomial regression' un degree'sini 5 yaparak bu durumu biz oluşturmuştuk.

,linear_train,linear_test
R2,0.997607,0.764990
mae,0.186213,0.665959
mse,0.062968,6.634847
rmse,0.250934,2.575820


### Multicollinearity
If there is a strong correlation between the independent variables, this situation is called multicollinearity.

Multicollinearity prevents my model from detecting important features.

In [140]:
def color_red(val):
    if abs(val) > 0.90 and abs(val) < 0.99:
        color = "red"
    else:
        color = "black"
    return f"color: {color}"

In [141]:
pd.DataFrame(poly_features).corr().style.applymap(color_red)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54
0,1.000000,0.054809,0.056648,0.967662,0.662160,0.603099,0.050844,0.046222,0.055626,0.912265,0.721105,0.650492,0.515275,0.466823,0.420083,0.048734,0.039665,0.028958,0.039409,0.855073,0.709761,0.630912,0.584542,0.516807,0.460056,0.437080,0.409771,0.364701,0.315533,0.046959,0.032900,0.017709,0.008777,0.020708,0.801581,0.680833,0.596879,0.588127,0.506564,0.447078,0.505741,0.462847,0.403394,0.346404,0.384146,0.373602,0.343453,0.286909,0.243084,0.045768,0.026080,0.007613,-0.004329,-0.007060,0.004975
1,0.054809,1.000000,0.354104,0.078644,0.681392,0.250204,0.966752,0.708347,0.352336,0.085030,0.548158,0.217652,0.738807,0.549689,0.259887,0.912163,0.738767,0.538912,0.296300,0.084899,0.473991,0.201889,0.615548,0.449913,0.218735,0.725701,0.596713,0.435679,0.221477,0.858447,0.728288,0.578239,0.405678,0.233682,0.082679,0.420642,0.191371,0.539663,0.389348,0.198233,0.616497,0.492912,0.353459,0.181827,0.697530,0.604193,0.480872,0.335472,0.178656,0.809579,0.705871,0.590660,0.441864,0.300963,0.183753
2,0.056648,0.354104,1.000000,0.076442,0.251706,0.710860,0.361405,0.846215,0.939677,0.087403,0.223935,0.586885,0.277229,0.649722,0.744348,0.348591,0.756600,0.848977,0.820403,0.093523,0.216804,0.523625,0.251371,0.545914,0.618126,0.270379,0.595505,0.681096,0.683016,0.329318,0.689047,0.792878,0.765061,0.697417,0.097150,0.214929,0.480870,0.244256,0.490294,0.547514,0.247790,0.509593,0.561287,0.560250,0.253301,0.548691,0.643265,0.634693,0.606379,0.308883,0.629623,0.744772,0.740034,0.661761,0.595177
3,0.967662,0.078644,0.076442,1.000000,0.655610,0.604786,0.074717,0.077064,0.085473,0.984478,0.754241,0.688295,0.515837,0.483686,0.441567,0.069934,0.072344,0.066860,0.071437,0.952132,0.772650,0.694074,0.615256,0.560653,0.504148,0.439513,0.430431,0.395637,0.346060,0.064975,0.064502,0.058043,0.047638,0.050892,0.914302,0.763650,0.675974,0.643078,0.569615,0.506698,0.533678,0.507202,0.453477,0.391984,0.386548,0.394148,0.376481,0.324503,0.277483,0.060771,0.055516,0.048174,0.037872,0.028539,0.031722
4,0.662160,0.681392,0.251706,0.655610,1.000000,0.593723,0.664399,0.485959,0.241570,0.627090,0.965952,0.613323,0.962962,0.777374,0.490596,0.631071,0.502268,0.356248,0.190696,0.594024,0.910381,0.596252,0.951173,0.762204,0.504128,0.905034,0.783817,0.618268,0.392485,0.597633,0.490224,0.372765,0.250397,0.138198,0.561829,0.853111,0.571575,0.906957,0.717061,0.487728,0.905252,0.771478,0.605065,0.401054,0.848834,0.767547,0.645128,0.482578,0.309573,0.567146,0.470411,0.371193,0.262030,0.169913,0.097704
5,0.603099,0.250204,0.710860,0.604786,0.593723,1.000000,0.253924,0.576143,0.663457,0.587286,0.627367,0.969091,0.538715,0.867956,0.937710,0.241838,0.504354,0.558658,0.564313,0.564874,0.627426,0.921108,0.583577,0.860898,0.909793,0.489275,0.786042,0.847498,0.826914,0.224767,0.448703,0.503393,0.481855,0.460299,0.542113,0.617954,0.873051,0.594081,0.826298,0.859458,0.536670,0.794702,0.826686,0.797872,0.443163,0.720646,0.800253,0.764051,0.714386,0.207342,0.399518,0.456294,0.445350,0.396279,0.372849
6,0.050844,0.966752,0.361405,0.074717,0.664399,0.253924,1.000000,0.717856,0.356512,0.081937,0.536793,0.221874,0.768358,0.554347,0.259786,0.985102,0.783297,0.552534,0.297371,0.082618,0.465376,0.206476,0.640765,0.454135,0.219410,0.787473,0.628876,0.439364,0.217841,0.955280,0.798072,0.614792,0.414209,0.232155,0.080961,0.413689,0.196065,0.561681,0.393295,0.199027,0.668950,0.518426,0.355656,0.179322,0.779714,0.658326,0.503108,0.333459,0.172715,0.921100,0.792812,0.646188,0.465716,0.303686,0.180208
7,0.046222,0.708347,0.846215,0.077064,0.485959,0.576143,0.717856,1.000000,0.834543,0.095289,0.411600,0.485136,0.543085,0.756910,0.620568,0.695496,0.977682,0.941437,0.743549,0.107359,0.378923,0.445148,0.471052,0.633268,0.518778,0.539793,0.763229,0.738939,0.573946,0.664420,0.935782,0.944307,0.819946,0.636391,0.116108,0.358969,0.420758,0.435308,0.565920,0.469426,0

In [142]:
# burada feature'lar arasındaki collinearity durumunu rahatlıkla görebiliyoruz.
# Kırmızı ile gösterilen değerler feature'lar arasında yüksek correlation olduğunu gösteriyor.

###  Cross-Validation
We do cross-validation to check whether the one-time scores we receive are consistent are not.

Cross-validation is only applied to the train set. If K-fold value is 5 , 1 fold is Validation fold and others Training Folds.

In [143]:
from sklearn.metrics import SCORERS
list(SCORERS.keys())
# Cross-validation içinde bulundan scoring parametresi için kullanabileceğimiz değerlere bakıyoruz.

['explained_variance',
 'r2',
 'max_error',
 'matthews_corrcoef',
 'neg_median_absolute_error',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_root_mean_squared_error',
 'neg_mean_poisson_deviance',
 'neg_mean_gamma_deviance',
 'accuracy',
 'top_k_accuracy',
 'roc_auc',
 'roc_auc_ovr',
 'roc_auc_ovo',
 'roc_auc_ovr_weighted',
 'roc_auc_ovo_weighted',
 'balanced_accuracy',
 'average_precision',
 'neg_log_loss',
 'neg_brier_score',
 'adjusted_rand_score',
 'rand_score',
 'homogeneity_score',
 'completeness_score',
 'v_measure_score',
 'mutual_info_score',
 'adjusted_mutual_info_score',
 'normalized_mutual_info_score',
 'fowlkes_mallows_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'jaccard',
 'jacca

In [144]:
from sklearn.model_selection import cross_validate

In [145]:
model = LinearRegression()
scores = cross_validate(model, X_train_scaled, y_train,
                        scoring = ["r2", "neg_mean_absolute_error", "neg_mean_squared_error", "neg_root_mean_squared_error"],
                        cv = 5)
# burada neg_ ifades değerin negatif olduğunu gösteriyor. cross_validate içinde neg_ olarak kullanmamız gerekiyor.
# Çünkü diğer parametreleri kabul etmiyor.
# Bu negatif değerleri pozitifmiş gibi yorumlayacağız.

In [146]:
scores

{'fit_time': array([0.00199294, 0.00199318, 0.00099683, 0.00199366, 0.00199342]),
 'score_time': array([0.00099659, 0.        , 0.00099635, 0.        , 0.00099635]),
 'test_r2': array([0.36790198, 0.87965778, 0.97756048, 0.98936937, 0.16659583]),
 'test_neg_mean_absolute_error': array([-1.27124712, -0.71046265, -0.39503284, -0.41897731, -1.47409318]),
 'test_neg_mean_squared_error': array([-19.20425854,  -2.71329198,  -0.62290055,  -0.28936097,
        -15.40985579]),
 'test_neg_root_mean_squared_error': array([-4.38226637, -1.64720733, -0.78924049, -0.53792283, -3.92553892])}

In [147]:
pd.DataFrame(scores, index=range(1,6))

,fit_time,score_time,test_r2,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_root_mean_squared_error
1,0.001993,0.000997,0.367902,-1.271247,-19.204259,-4.382266
2,0.001993,0.000000,0.879658,-0.710463,-2.713292,-1.647207
3,0.000997,0.000996,0.977560,-0.395033,-0.622901,-0.789240
4,0.001994,0.000000,0.989369,-0.418977,-0.289361,-0.537923
5,0.001993,0.000996,0.166596,-1.474093,-15.409856,-3.925539


In [148]:
# Yukarıdaki dataframe için fit_time ve score_time ile işimiz yok diğerlerini kullanacağız.
scores = pd.DataFrame(scores, index=range(1,6))
scores.iloc[:, 2:].mean()
# burada cross_validation sonrası ortaya çıkan tüm değerlerin ortalamasını alarak genel bir score üretiyoruz.

test_r2                             0.676217
test_neg_mean_absolute_error       -0.853963
test_neg_mean_squared_error        -7.647934
test_neg_root_mean_squared_error   -2.256435
dtype: float64

In [149]:
train_val(y_train, y_train_pred, y_test, y_pred, "linear")

,linear_train,linear_test
R2,0.997607,0.764990
mae,0.186213,0.665959
mse,0.062968,6.634847
rmse,0.250934,2.575820


## Ridge Regression 

In [150]:
from sklearn.linear_model import Ridge

In [151]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def train_val(y_train, y_train_pred, y_test, y_pred, name):
    
    scores = {name+"_train": {"R2" : r2_score(y_train, y_train_pred),
    "mae" : mean_absolute_error(y_train, y_train_pred),
    "mse" : mean_squared_error(y_train, y_train_pred),                          
    "rmse" : np.sqrt(mean_squared_error(y_train, y_train_pred))},
    
    name+"_test": {"R2" : r2_score(y_test, y_pred),
    "mae" : mean_absolute_error(y_test, y_pred),
    "mse" : mean_squared_error(y_test, y_pred),
    "rmse" : np.sqrt(mean_squared_error(y_test, y_pred))}}
    
    return pd.DataFrame(scores)

In [152]:
ridge_model = Ridge(alpha=1, random_state=42) #alpha=1 default value.

In [153]:
ridge_model.fit(X_train_scaled, y_train)

Ridge(alpha=1, random_state=42)

In [154]:
y_pred = ridge_model.predict(X_test_scaled)
y_train_pred = ridge_model.predict(X_train_scaled)

In [155]:
rs = train_val(y_train, y_train_pred, y_test, y_pred, "ridge")
rs

,ridge_train,ridge_test
R2,0.988677,0.982511
mae,0.338149,0.482446
mse,0.297970,0.493743
rmse,0.545866,0.702669


#### Cross Validation (alpha = 1) 

In [156]:
from sklearn.model_selection import cross_validate

In [157]:
model = Ridge(alpha=1, random_state=42)
scores = cross_validate(model, X_train_scaled, y_train,
                        scoring = ["r2", "neg_mean_absolute_error", "neg_mean_squared_error", "neg_root_mean_squared_error"],
                        cv = 5)

In [158]:
pd.DataFrame(scores, index=range(1, 6))

,fit_time,score_time,test_r2,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_root_mean_squared_error
1,0.000998,0.000000,0.988913,-0.405789,-0.336840,-0.580379
2,0.000000,0.000997,0.969951,-0.551469,-0.677493,-0.823099
3,0.000000,0.000996,0.950978,-0.552320,-1.360795,-1.166531
4,0.000997,0.000000,0.991259,-0.365106,-0.237919,-0.487769
5,0.000998,0.000000,0.986854,-0.343011,-0.243072,-0.493023


In [159]:
scores = pd.DataFrame(scores, index=range(1,6))
scores.iloc[:,2:].mean()

test_r2                             0.977591
test_neg_mean_absolute_error       -0.443539
test_neg_mean_squared_error        -0.571224
test_neg_root_mean_squared_error   -0.710160
dtype: float64

In [160]:
train_val(y_train, y_train_pred, y_test, y_pred, "ridge")

,ridge_train,ridge_test
R2,0.988677,0.982511
mae,0.338149,0.482446
mse,0.297970,0.493743
rmse,0.545866,0.702669


#### Choosing best alpha value by RidgeCV

In [161]:
from sklearn.linear_model import RidgeCV
# RidgeCV, ileride öğreneceğimiz GridSearch ile aynı işlemi yapar ancak sadece Ridge için yapar.
# Genel olarak GridSearch kullanacağız ancak burada göstermiş olduk.

In [162]:
alpha_space = np.linspace(0.01, 1, 100)
alpha_space
# Burada alpha için değer aralığı oluşturduk. RidgeCv kullanırken bu değerlerden en uygunu bulmasını isteyeceğiz.

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99,
       1.  ])

In [163]:
ridge_cv_model = RidgeCV(alphas = alpha_space, cv=5, scoring="neg_root_mean_squared_error")
# Ridge_CV' de scoring için sadece bir tane error metric' i yazabiliyoruz. Daha fazlasına müsaade edilmiyor.

In [164]:
ridge_cv_model.fit(X_train_scaled, y_train)

RidgeCV(alphas=array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99,
       1.  ]),
        cv=5, scoring='neg_root_mean_squared_error')

In [165]:
ridge_cv_model.alpha_
# Verdiğimiz değerler içinden en uygun alpha değeri 0.02 çıktı.

0.02

In [166]:
ridge_cv_model.best_score_
# 0.02 alpha değeri için en iyi R2_score budur.

-0.6530406519554066

In [167]:
y_pred = ridge_cv_model.predict(X_test_scaled)
y_train_pred = ridge_cv_model.predict(X_train_scaled)

In [168]:
rcs = train_val(y_train, y_train_pred, y_test, y_pred, "ridge_cv")
rcs

,ridge_cv_train,ridge_cv_test
R2,0.994491,0.983643
mae,0.244842,0.442087
mse,0.144977,0.461803
rmse,0.380758,0.679561


In [169]:
pd.concat([rs, rcs], axis=1)
# Burada alpha=1 ve alpha=0.02 için ortaya çıkan sonuçların karşılatırmasın görüyoruz.

,ridge_train,ridge_test,ridge_cv_train,ridge_cv_test
R2,0.988677,0.982511,0.994491,0.983643
mae,0.338149,0.482446,0.244842,0.442087
mse,0.297970,0.493743,0.144977,0.461803
rmse,0.545866,0.702669,0.380758,0.679561


## Lasso Regression

In [170]:
from sklearn.linear_model import Lasso, LassoCV

In [171]:
lasso_model = Lasso(alpha=1, random_state=42) # alpha =1 default değer. Önce default değer ile deniyoruz. Sonra CV yapacaz.
lasso_model.fit(X_train_scaled, y_train)

Lasso(alpha=1, random_state=42)

In [172]:
y_pred = lasso_model.predict(X_test_scaled)
y_train_pred = lasso_model.predict(X_train_scaled)

In [173]:
lss = train_val(y_train,y_train_pred, y_test, y_pred, "lasso")
lss

,lasso_train,lasso_test
R2,0.919650,0.918590
mae,1.018355,1.017959
mse,2.114491,2.298390
rmse,1.454129,1.516044


In [174]:
pd.concat([rs, rcs, lss], axis=1)
# Burada ridge ve lasso değerlerinin karşılaştırmasını görüyoruz.

,ridge_train,ridge_test,ridge_cv_train,ridge_cv_test,lasso_train,lasso_test
R2,0.988677,0.982511,0.994491,0.983643,0.919650,0.918590
mae,0.338149,0.482446,0.244842,0.442087,1.018355,1.017959
mse,0.297970,0.493743,0.144977,0.461803,2.114491,2.298390
rmse,0.545866,0.702669,0.380758,0.679561,1.454129,1.516044


### Cross Validation (alpha = 1) 

In [175]:
model = Lasso(alpha=1, random_state=42)
scores = cross_validate(model,X_train_scaled, y_train,
                        scoring = ["r2", "neg_mean_absolute_error", "neg_mean_squared_error", "neg_root_mean_squared_error"],
                        cv=5)

In [176]:
pd.DataFrame(scores, index=range(1,6))

,fit_time,score_time,test_r2,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_root_mean_squared_error
1,0.001993,0.000996,0.924246,-1.155268,-2.301550,-1.517086
2,0.000997,0.000000,0.955624,-0.656358,-1.000530,-1.000265
3,0.000997,0.000000,0.888327,-1.072936,-3.099952,-1.760668
4,0.000000,0.000998,0.896199,-1.255165,-2.825429,-1.680901
5,0.000995,0.000000,0.903545,-1.056538,-1.783470,-1.335466


In [177]:
scores = pd.DataFrame(scores, index = range(1,6))
scores.iloc[:,2:].mean()

test_r2                             0.913588
test_neg_mean_absolute_error       -1.039253
test_neg_mean_squared_error        -2.202186
test_neg_root_mean_squared_error   -1.458877
dtype: float64

In [178]:
train_val(y_train, y_train_pred, y_test, y_pred, "lasso")

,lasso_train,lasso_test
R2,0.919650,0.918590
mae,1.018355,1.017959
mse,2.114491,2.298390
rmse,1.454129,1.516044


### Choosing best alpha value by LassoCV

In [179]:
lasso_cv_model = LassoCV(alphas=alpha_space, cv=5, max_iter=100000, random_state=42)
# Normalde default olarak 1000 defa iterasyon yapıyor. 
# Ancak bu değer düşük olduğu için biz 100 000 olarak değiştirdik.

In [180]:
lasso_cv_model.fit(X_train_scaled, y_train)

LassoCV(alphas=array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99,
       1.  ]),
        cv=5, max_iter=100000, random_state=42)

In [181]:
lasso_cv_model.alpha_
# Verdiğimiz aralıkta lasso için e uygun alpha değeri 0.01 imiş.

0.01

In [182]:
y_pred = lasso_cv_model.predict(X_test_scaled)
y_train_pred = lasso_cv_model.predict(X_train_scaled)

In [183]:
lcs = train_val(y_train, y_train_pred, y_test, y_pred, "lasso_cv")
lcs

,lasso_cv_train,lasso_cv_test
R2,0.988824,0.986295
mae,0.339065,0.440975
mse,0.294098,0.386919
rmse,0.542308,0.622028


In [184]:
pd.concat([rs, rcs, lss, lcs], axis=1)
# Test sonuçlarını karşılaştırdığımızda Lasso_cv'nin en uygun olduğunu görüyoruz.

,ridge_train,ridge_test,ridge_cv_train,ridge_cv_test,lasso_train,lasso_test,lasso_cv_train,lasso_cv_test
R2,0.988677,0.982511,0.994491,0.983643,0.919650,0.918590,0.988824,0.986295
mae,0.338149,0.482446,0.244842,0.442087,1.018355,1.017959,0.339065,0.440975
mse,0.297970,0.493743,0.144977,0.461803,2.114491,2.298390,0.294098,0.386919
rmse,0.545866,0.702669,0.380758,0.679561,1.454129,1.516044,0.542308,0.622028


## Elastic Net

In [185]:
from sklearn.linear_model import ElasticNet, ElasticNetCV

In [186]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def train_val(y_train, y_train_pred, y_test, y_pred, name):
    
    scores = {name+"_train": {"R2" : r2_score(y_train, y_train_pred),
    "mae" : mean_absolute_error(y_train, y_train_pred),
    "mse" : mean_squared_error(y_train, y_train_pred),                          
    "rmse" : np.sqrt(mean_squared_error(y_train, y_train_pred))},
    
    name+"_test": {"R2" : r2_score(y_test, y_pred),
    "mae" : mean_absolute_error(y_test, y_pred),
    "mse" : mean_squared_error(y_test, y_pred),
    "rmse" : np.sqrt(mean_squared_error(y_test, y_pred))}}
    
    return pd.DataFrame(scores)

In [187]:
elastic_model = ElasticNet(alpha=1, l1_ratio=0.5, random_state=42)
# Burada alpha = 1 ve l1_ratio = 0.5 değerleri default değerler. 
# Elastic Net için ayarlanması gereken 2 adet hyperparameter olduğunu görüyoruz.
# burada l1_ratio = 0.5 demek %50 lasso %50 ridge kullanılacak demektir.
# l1_ratio = 1 olursa sadece lasso, l1_ratio = 0 olursa sadece Ridge kullanılacak demektir.

elastic_model.fit(X_train_scaled, y_train)

ElasticNet(alpha=1, random_state=42)

In [188]:
y_pred = elastic_model.predict(X_test_scaled)
y_train_pred = elastic_model.predict(X_train_scaled)

In [189]:
es = train_val(y_train, y_train_pred, y_test, y_pred, "elastic")
es
#Burada default değerler kullanıldığı için sonuçları çok iyi değil.

,elastic_train,elastic_test
R2,0.896126,0.902016
mae,1.248480,1.200654
mse,2.733565,2.766287
rmse,1.653350,1.663216


### ElasticNet Cross-Validation (Default parameters : alpha = 1, and l1_ratio = 0.5)

In [190]:
from sklearn.model_selection import cross_validate

In [191]:
model = ElasticNet(alpha=1, l1_ratio=0.5, random_state=42)
scores = cross_validate(model, X_train_scaled, y_train,
                        scoring=["r2", "neg_mean_absolute_error", "neg_mean_squared_error", "neg_root_mean_squared_error"],
                        cv=5)

In [192]:
scores = pd.DataFrame(scores, index=range(1,6))
scores.iloc[:, 2:]

,test_r2,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_root_mean_squared_error
1,0.915497,-1.255282,-2.567362,-1.602299
2,0.928138,-0.960011,-1.620225,-1.272881
3,0.855390,-1.373843,-4.014250,-2.003559
4,0.875648,-1.431847,-3.384817,-1.839787
5,0.840262,-1.449269,-2.953597,-1.718603


In [193]:
scores.iloc[:, 2:].mean()
# cross validation sonrası score'umuz 0.88

test_r2                             0.882987
test_neg_mean_absolute_error       -1.294050
test_neg_mean_squared_error        -2.908050
test_neg_root_mean_squared_error   -1.687426
dtype: float64

In [194]:
train_val(y_train, y_train_pred, y_test, y_pred, "elastic")

,elastic_train,elastic_test
R2,0.896126,0.902016
mae,1.248480,1.200654
mse,2.733565,2.766287
rmse,1.653350,1.663216


### Choosing best alpha and l1_ratio values by ElasticNetCV

In [195]:
alpha_space = np.linspace(0.01, 1, 100)
alpha_space

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99,
       1.  ])

In [196]:
l1_ratio_space = np.linspace(0.01, 1, 100)

In [197]:
elastic_cv_model = ElasticNetCV(alphas=alpha_space, l1_ratio=l1_ratio_space, cv = 5,
                                max_iter=100000, random_state=42)

In [198]:
elastic_cv_model.fit(X_train_scaled, y_train)

ElasticNetCV(alphas=array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66...
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99,
       1.  ]),
             max_iter=100000, random_state=42)

In [199]:
elastic_cv_model.alpha_
# Verdiğimiz aralıklarda en uygun alpha değeri 0.01 çıktı.

0.01

In [200]:
elastic_cv_model.l1_ratio_
# en uygun l1_ratio değeri de 1.0 çıktı yani bize sadece lasso kullan  diyor.
# l1 =lasso, l2 =ridge

1.0

In [201]:
y_pred = elastic_cv_model.predict(X_test_scaled)
y_train_pred = elastic_cv_model.predict(X_train_scaled)

In [202]:
ecs = train_val(y_train, y_train_pred, y_test, y_pred, "elastic_cv")
ecs

,elastic_cv_train,elastic_cv_test
R2,0.988824,0.986295
mae,0.339065,0.440975
mse,0.294098,0.386919
rmse,0.542308,0.622028


In [203]:
pd.concat([es, ecs], axis=1)
# crossvalidation sonrası test scoru'muz çok arttı.(0.98)

,elastic_train,elastic_test,elastic_cv_train,elastic_cv_test
R2,0.896126,0.902016,0.988824,0.986295
mae,1.248480,1.200654,0.339065,0.440975
mse,2.733565,2.766287,0.294098,0.386919
rmse,1.653350,1.663216,0.542308,0.622028


## Grid Search 

In [204]:
from sklearn.model_selection import GridSearchCV
# RidgeCV, LassoCV, ElasticCV yerine Grid Search uygulayacağız. Grid Search tüm modellerde kullanılabilir.
# Bu işlem data boyutuna bağlı olarak çok uzun zaman alabilir.

In [205]:
elastic_model = ElasticNet(max_iter=10000, random_state=42)

In [206]:
param_grid = {"alpha": [0.01, 0.012, 0.2, 0.5, 0.6, 0.7, 1],
              "l1_ratio": [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]}

# Burada alpha ve l1_ratio için çok daha fazla değer verebilirdik ancak uzun sürmesini istemiyoruz.

In [207]:
grid_model = GridSearchCV(estimator=elastic_model, 
                          param_grid=param_grid, 
                          scoring="neg_root_mean_squared_error",
                          cv = 5, verbose=2)

# verbose = 2 (her 2 iterasyonda 1 bana sonuçları göstersin istiyorum.)
# estimator = elastic_model (Elastic Net seçiyoruz ki lasso ve ridge ikisi de değerlendirilsin.)

In [208]:
grid_model.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 49 candidates, totalling 245 fits
[CV] END ...........................alpha=0.01, l1_ratio=0.1; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.1; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.1; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.1; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.1; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.5; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.5; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.5; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.5; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.5; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.7; total time=   0.0s
[CV] END ...........................alpha=0.01,

[CV] END ...........................alpha=0.6, l1_ratio=0.95; total time=   0.0s
[CV] END ...........................alpha=0.6, l1_ratio=0.95; total time=   0.0s
[CV] END ...........................alpha=0.6, l1_ratio=0.99; total time=   0.0s
[CV] END ...........................alpha=0.6, l1_ratio=0.99; total time=   0.0s
[CV] END ...........................alpha=0.6, l1_ratio=0.99; total time=   0.0s
[CV] END ...........................alpha=0.6, l1_ratio=0.99; total time=   0.0s
[CV] END ...........................alpha=0.6, l1_ratio=0.99; total time=   0.0s
[CV] END ..............................alpha=0.6, l1_ratio=1; total time=   0.0s
[CV] END ..............................alpha=0.6, l1_ratio=1; total time=   0.0s
[CV] END ..............................alpha=0.6, l1_ratio=1; total time=   0.0s
[CV] END ..............................alpha=0.6, l1_ratio=1; total time=   0.0s
[CV] END ..............................alpha=0.6, l1_ratio=1; total time=   0.0s
[CV] END ...................

GridSearchCV(cv=5, estimator=ElasticNet(max_iter=10000, random_state=42),
             param_grid={'alpha': [0.01, 0.012, 0.2, 0.5, 0.6, 0.7, 1],
                         'l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]},
             scoring='neg_root_mean_squared_error', verbose=2)

In [209]:
grid_model.best_params_
# Grid Search sonrası ulaştığımız best parametre değerleri daha öncekilerle(LassoCV, RidgeCV) aynı çıktı.

{'alpha': 0.01, 'l1_ratio': 1}

In [210]:
grid_model.best_score_
# en iyi score 0.59 imiş.

-0.5908719878682451

### Using Best Hyperparameters from GridSearch

In [211]:
y_pred = grid_model.predict(X_test_scaled)
y_train_pred = grid_model.predict(X_train_scaled)

In [213]:
train_val(y_train, y_train_pred, y_test, y_pred, "GridSearch")
# Grid Search işlemi sonrası tes ve train datasını karşılaştırıyoruz. Gayet iyi.
# Test score'umuz % 98.6 çıktı.

,GridSearch_train,GridSearch_test
R2,0.988824,0.986295
mae,0.339065,0.440975
mse,0.294098,0.386919
rmse,0.542308,0.622028


## Final Model

In [214]:
final_scaler = StandardScaler()
X_scaled = final_scaler.fit_transform(poly_features)
# Final model'de artık datanın hepsini kullanıyoruz.(poly features)

In [215]:
from sklearn.linear_model import Lasso
final_model = Lasso(alpha=0.01)
# alpha = 0.01 ile en iyi sonucu alacağımızı gridsearch sonrası öğrenmiştik.

In [216]:
final_model.fit(X_scaled, y)
# Final model'i tüm data üzerinden eğittik.

Lasso(alpha=0.01)

In [217]:
#Bu modelin son halini saklıyoruz. Belki bir websitesi üzerinden hizmet verecek...
# Kullanıcı burada radyo, tv, gazete için reklam harcaması girecek ve satıiş miktarındaki değişimleri görecek.
# Bu verilere göre karar alacak.
# Bütün modellerde bu şekilde yapacağız.
# Bu modelleri deploy etmeyi Machine Learning - Deployment kısmında göreceğiz.
# Sürekli yeni data girişi varsa o dataları ekleyip modeli refresh ederek yeniden deploy etmek gerekir.